In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import sys

In [3]:
sys.path.append('../src')

In [4]:
df = pd.read_parquet('../data/AI_Human_cleaned.parquet')

In [13]:
from utils.text_features import FeatureTextExtraction

In [25]:
feature_extractor = FeatureTextExtraction()

In [26]:
df_final = feature_extractor.fit_transform(df)

In [27]:
df_final

,dim1,dim2,dim3,dim4,dim5
0,-0.120500,-0.189132,-0.020649,0.231255,0.089698
1,-0.018106,-0.030622,0.239225,-0.197513,0.029345
2,-0.073475,-0.054289,0.149792,-0.241656,-0.032967
3,-0.218399,-0.342125,-0.273788,0.015312,-0.091625
4,-0.026885,-0.143897,0.078570,0.298427,0.014897
...,...,...,...,...,...
39995,-0.063457,-0.052466,0.167950,-0.143063,0.240022
39996,0.780618,-0.055180,-0.101664,0.003742,-0.034919
39997,-0.258774,-0.427603,-0.358374,0.072386,-0.126380
39998,-0.054871,0.233206,0.115173,-0.107147,-0.048017
